<a href="https://colab.research.google.com/github/AIexndr/API/blob/main/%D0%9B%D0%B5%D0%BA%D1%86%D0%B8%D1%8F_2_%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D0%B2%D0%B0%D0%BA%D0%B0%D0%BD%D1%81%D0%B8%D0%B9_HH_%D0%94%D0%97_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import re
import pandas as pd
import numpy as np

# функция аргументы - адрес-хедер-парам 
def get_body(address, headers, params):
    page = requests.get(address, headers=headers, params=params)
    result = page.text
    pprint(result)
    link = page.url
# на выходе - линк и текст на стр
    return link, result

#функция вакансий
def get_vacs_HH(page):
    # делаем пустые списки
    list_names = []
    list_links = []
    list_minzp = []
    list_maxzp = []
    list_sources = []
    source = 'HeadHunter'
    soup = bs(page, 'lxml')
# библиотекой б-с  ищем через 'div'
    block_vacs = soup.find('div', {'class' : 'vacancy-serp'})

    for vac in block_vacs:
        list_sources.append(source)
        head = vac.find('div', { 'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})
        if head is None:
            continue
        
        a = head.find('a')

        list_names.append(a.getText())
        
        list_links.append(a['href'])
         
        zp = vac.find ('div', { 'class' : 'vacancy-serp-item__compensation'})
        if zp is None:
            minzp = 'Нет данных'
            maxzp = ''
            list_minzp.append(minzp)
            list_maxzp.append(maxzp)
             
            continue
        zp_full = zp.getText().replace('\xa0','')
         
        if zp_full[0:2] == 'от':
            minzp = re.findall('\d*', zp_full)[3]
            maxzp = ''
            list_minzp.append(minzp)
            list_maxzp.append(maxzp)
             
        elif zp_full[0:2] == 'до':
            minzp = ''
            maxzp = re.findall('\d*', zp_full)[3]
            list_minzp.append(minzp)
            list_maxzp.append(maxzp)
             
        else:
            list_zp = re.split('\-', zp_full)
            minzp = list_zp[0]
            maxzp = list_zp[1]
            list_minzp.append(minzp)
            list_maxzp.append(maxzp)
     # на выходе списки названия, линков, зарп мин, макс, источник        
    return list_names, list_links, list_minzp, list_maxzp, list_sources

#назв вакансии
name_vac = 'Data Scientist'
# страницы
pages= list('1,2,3,4,5')
num_pages = [pages[x] for x in range(0, len(pages), 2)]
#адрес сайта
address = 'https://hh.ru/search/vacancy'
list_num=[]
#хедер
headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36 OPR/63.0.3368.71"}

names = []
linki = []
mins = []
maxs = []
istochniki = []

for page in num_pages:
    params = {"L_is_autosearch": "false", "area": "1", "clusters": "true", "text": name_vac, "page": page}
    url, body = get_body(address, headers, params)
    name, vlink, min, max, istochnik = get_vacs_HH(body)
    names = names+name
    linki = linki+vlink
    mins = mins + min
    maxs = maxs + max
    istochniki = istochniki + istochnik
# данные в види дата-сета пандас
dataframe_na_export = pd.DataFrame(list(zip(names, linki, mins, maxs, istochniki)), columns=['Имя', 'Ссылка',
                                                                                             'Мин. зп','Макс. зп',
#запись в таблицу Эксель на диск                                                                                             'Источник'])
dataframe_na_export.to_excel('output1.xlsx')
